    Should I ever need to use client.annotation.get_annotation ? (JSON rep of data)

In [ ]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
def class_spitter(df):
    classes = np.unique(df.classification_system)
    cellarray = []
    for i in range(len(classes)):
        new = df.query(f"classification_system in @classes[{i}]")
        cellarray.append(new)
    return cellarray

def type_spitter(df):
    types = np.unique(df.cell_type)
    cellarray = []
    for i in range(len(types)):
        new = df.query(f"cell_type in @types[{i}]")
        cellarray.append(new)
    return cellarray

In [ ]:
def Eucdistance_split(pre,post):
    x_pre,y_pre,z_pre = np.array(pre.pt_position_x)*4,np.array(pre.pt_position_y)*4,np.array(pre.pt_position_z)*40
    x_pos,y_pos,z_pos = np.array(post.pt_position_x)*4,np.array(post.pt_position_y)*4,np.array(post.pt_position_z)*40
    d = np.zeros(len(post))
    for i in range(len(post)):
        # divide by 1000 to convert to microns
        d[i] = np.sqrt((x_pre-x_pos[i])**2 + (y_pre-y_pos[i])**2 + (z_pre-z_pos[i])**2) / 1000.
    return d

def Raddistance_split(pre,post):
    x_pre,z_pre = np.array(pre.pt_position_x)*4,np.array(pre.pt_position_z)*40
    x_pos,z_pos = np.array(post.pt_position_x)*4,np.array(post.pt_position_z)*40
    d = np.zeros(len(post))
    for i in range(len(post)):
        # divide by 1000 to convert to microns
        d[i] = np.sqrt((x_pre-x_pos[i])**2 + (z_pre-z_pos[i])**2) / 1000.
    return d

In [ ]:
client = CAVEclient(global_only=True)
# https://annotationframeworkclient.readthedocs.io/en/stable/guide/authentication.html#new-token
client = CAVEclient('minnie65_phase3_v1')
client.info.get_datastack_info()

In [ ]:
#client.materialize.get_tables()

In [ ]:
df = client.materialize.query_table('allen_v1_column_types_slanted',split_positions=True)
print(len(df))
df[0:1]

In [ ]:
# start cell
root_id = [864691135428608048]
boy = df.query("pt_root_id in @root_id")
# post-synaptic partners of starter cell
syn_unfiltered = client.materialize.query_table('synapses_pni_2',filter_equal_dict={'pre_pt_root_id':root_id[0]})

In [ ]:
unique_syn = np.unique(syn_unfiltered.post_pt_root_id)
print("There are {0:g} synaptic connections from {1:g} cells (unfiltered for mergers).".format(len(syn_unfiltered),len(unique_syn)))

    Now to see how many of those cells are nuclei vs glial cells...

In [ ]:
correct_soma_table = client.info.get_datastack_info()['soma_table']
nuclei_unmasked = client.materialize.query_table(correct_soma_table,split_positions=True)
# new df of just neurons (no glial cells)
nuclei = nuclei_unmasked.query('cell_type == "neuron"').reset_index(drop=True)
# new column saying how many neurons have the same root_id
nuclei['num_soma'] = nuclei.groupby('pt_root_id').transform('count')['valid']
# mask the df to throw out merged nuclei (same root_id being assigned to multiple neurons)
mask_nuclei = nuclei['num_soma'] < 2
nuclei_full = nuclei[mask_nuclei].reset_index(drop=True)

In [ ]:
unique_nuc = np.unique(nuclei_full.pt_root_id)
soma_full = client.materialize.query_table('allen_soma_coarse_cell_class_model_v1',filter_in_dict = {'pt_root_id':unique_nuc},
                                          split_positions=True)

In [ ]:
print(len(soma_full))
soma_full[0:1]

In [ ]:
syn_nuc = syn_unfiltered.query("post_pt_root_id in @unique_nuc").reset_index(drop=True)

In [ ]:
syn_nuc['num_syn'] = syn_nuc.groupby('post_pt_root_id').transform('count')['valid']
print(len(syn_nuc))
syn_nuc[0:1]

In [ ]:
syn_nuc.rename(columns={'post_pt_root_id':'pt_root_id'}, inplace=True)

In [ ]:
main_duplicates = pd.merge(soma_full,syn_nuc,on='pt_root_id',how='outer')[0:61847] # any cells after 61847 are not in the soma table
new_left = pd.merge(soma_full,syn_nuc,on='pt_root_id',how='left')
main = new_left.drop_duplicates(subset='pt_root_id', keep='first').reset_index(drop=True)

In [ ]:
main = main.drop(columns=['valid_y', 'pre_pt_supervoxel_id', 'post_pt_supervoxel_id', 'pre_pt_position', 'post_pt_position'])

In [ ]:
main['pre_pt_root_id'] = main['pre_pt_root_id'].fillna(0)
main['num_syn'] = main['num_syn'].fillna(0)

In [ ]:
#weird = [864691136311986237]
#new_outer.query('pt_root_id in @weird')

In [ ]:
#good = [864691134884758266]
#new_outer.query('pt_root_id in @good')

In [ ]:
#this takes forever
#%%time
main['d'] = Eucdistance_split(boy,main)
main['r'] = Raddistance_split(boy,main)

In [ ]:
main[0:1]

In [ ]:
unique_syn_nuc = np.unique(syn_nuc.pt_root_id)
syn = main.query('pt_root_id in @unique_syn_nuc')
nonsyn = main.query('pt_root_id not in @unique_syn_nuc')

In [ ]:
syn_class = class_spitter(syn)
nonsyn_class = class_spitter(nonsyn)

In [ ]:
fig, ax = plt.subplots(2,1)
fig.set_size_inches(17,11)

ax[0].hist(syn_class[0]['r'], 200, label='Synaptic Partners', density=False, alpha=1.0)
ax[0].hist(nonsyn_class[0]['r'], 300, label='Non-Synaptic Partners', density=False, alpha=0.6)
ax[0].set_title("Radial Distance to Excitatory Neurons (density=false)", fontsize=20)
ax[0].set_xlabel(r'$\mu$m', fontsize=14)
ax[0].legend()
ax[0].grid()

ax[1].hist(syn_class[1]['r'], 100, label='Synaptic Partners', density=False, alpha=1.0)
ax[1].hist(nonsyn_class[1]['r'], 200, label='Non-Synaptic Partners', density=False, alpha=0.6)
ax[1].set_title("Radial Distance to Inhibitory Neurons (density=false)", fontsize=20)
ax[1].set_xlabel(r'$\mu$m', fontsize=14)
ax[1].legend()
ax[1].grid()

ax[0].set_xlim(-10,1000)
ax[1].set_xlim(-10,1000)
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1)
fig.set_size_inches(17,11)

ax[0].hist(syn_class[0]['r'], 100, label='Synaptic Partners', density=True, alpha=0.6)
ax[0].hist(nonsyn_class[0]['r'], 400, label='Non-Synaptic Partners', density=True, alpha=0.6)
ax[0].set_title("Radial Distance to Excitatory Neurons (density=true)", fontsize=20)
ax[0].set_xlabel(r'$\mu$m', fontsize=14)
ax[0].legend()
ax[0].grid()

ax[1].hist(syn_class[1]['r'], 60, label='Synaptic Partners', density=True, alpha=0.6)
ax[1].hist(nonsyn_class[1]['r'], 300, label='Non-Synaptic Partners', density=True, alpha=0.6)
ax[1].set_title("Radial Distance to Inhibitory Neurons (density=true)", fontsize=20)
ax[1].set_xlabel(r'$\mu$m', fontsize=14)
ax[1].legend()
ax[1].grid()

ax[0].set_xlim(-10,1000)
ax[1].set_xlim(-10,1000)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
#weight = syn.eval("num_syn / 40").rename("weight")
g = sns.scatterplot(data=syn,x='r',y='num_syn',hue='classification_system', size='size', sizes=(20, 200));
#palette = 'deep'
g.set(xlim=(0, 250));
g.set(ylim=(0, 20));
g.set_title('Radial Distance vs number of synapses per soma', fontsize=18);

In [ ]:
#, binwidth=(2, .5)
beep = sns.displot(syn, x='r', y='num_syn', cbar=True, height=6, aspect=3);
beep.set(xlim=(0, 250));
beep.set(ylim=(0, 20));
beep.fig.suptitle('Radial distance vs number of synapses per soma', fontsize=16);

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
#weight = syn.eval("num_syn / 40").rename("weight")
g = sns.scatterplot(data=main, x=main.pt_position_x/4., y=main.pt_position_z/40., 
                    hue='pre_pt_root_id', 
                    style = 'classification_system', size='num_syn', palette = 'deep');#, sizes=(20, 200));
sns.scatterplot(x=boy.pt_position_x/4,
               y=boy.pt_position_z/40,
               marker='*',color='r',s=300);
#g.set(xlim=(0, 250));
#g.set(ylim=(0, 20));

In [ ]:
sns.set_theme(style="ticks")

sns.jointplot(x=syn.pt_position_x/4, 
              y=syn.pt_position_z/40, 
              kind='hex',color="#4CB391", height=8)#, marginal_kws=dict(bins=35, fill=False))

sns.scatterplot(x=boy.pt_position_x/4,
               y=boy.pt_position_z/40,
               marker='*',color='r',s=300);

In [ ]:
#sns.set(rc={'figure.figsize':(20,8)})
#fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
#plt.hisplot(syn_class[1], x="r", hue="cell_type", element="step", ax=ax1, height=7, aspect=2)
#plt.hisplot(syn_class[0], x="r", hue="cell_type", element="step", ax=ax2, height=7, aspect=2)
#g.fig.suptitle('Radial Distance of Excitatory Post-Synaptic Partners, Categorized by Cell Type', fontsize=16);
#g.fig.suptitle('Radial Distance of Inhibitory Post-Synaptic Partners, Categorized by Cell Type', fontsize=16);
#g.set(xlim=(0, 400));

fig, axs = plt.subplots(2, 2, figsize=(20, 14))

sns.histplot(data=syn_class[0], x="r", hue='cell_type', element='step', ax=axs[0,0]).set(xlim=(0,400));
sns.histplot(data=syn_class[1], x="r", hue='cell_type', element='step', ax=axs[0,1]).set(xlim=(0,400));
sns.histplot(data=nonsyn_class[0], x="r", hue='cell_type', element='step', ax=axs[1,0]).set(xlim=(0,400));
sns.histplot(data=nonsyn_class[1], x="r", hue='cell_type', element='step', ax=axs[1,1]).set(xlim=(0,400));

In [ ]:
import plotly_express as px

x = syn['pt_position_x'] / 4
y = syn['pt_position_y'] / 4
z = syn['pt_position_z'] / 40

bep = px.scatter_3d(syn, x=x, y=y, z=z,
              color='classification_system', size='num_syn')
#px.scatter_3d(boy,x=boy.pt_position_x/4,y=boy.pt_position_y/4,z=boy.pt_position_z/40,symbol='pt_root_id')
bep.show()

In [ ]:
x_main = main['pt_position_x'] / 4
y_main = main['pt_position_y'] / 4
z_main = main['pt_position_z'] / 40

g = px.scatter_3d(main, x=x_main, y=y_main, z=z_main,
              color='pre_pt_root_id', opacity=.1, range_color = [0,1], size=np.ones(len(main)))

g.show()

 calculate angle to cells from pre_synaptic cell
 
 calculate probability of connection, based on type & distance & bla
 
 "What fraction of cells of that type does the starter cell synapse onto (i.e. the starter cell is presynaptic and the target cells are postsynaptic).
 
 What is the distribution of total number of synapses from the starter cell onto those cells per cell type
 
 What fraction of cells of that type does the starter cell synapse onto (i.e. the starter cell is presynaptic and the target cells are postsynaptic)
 
 What is the distribution of total number of synapses from the starter cell onto those cells per cell type"